# Evaluation
## 1. 성능 평가 지표 (Evaluation Metric)
- 분류
    - 회귀인 경우
        - 실제값과 예측값의 오차 평균값에 기반
    - 분류인 경우 
        - 정확도 (Accuracy)
        - 오차행렬 (Confusion Matrix)
        - 정밀도 (Precision)
        - 재현율 (Recall)
        - F1 스코어
        - ROC AUC

### (1) 정확도 (Accuracy)
```
정확도 = 예측 결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
```
- 실제 데이터에서 예측 데이터가 얼마나 같은지 판단
- 불균형한 label data set에서는 성능 수치로 사용되면 X
    - 데이터 분포도가 균일하지 않은 경우 정확도가 높은 수치가 나타날 수 있음 
- 한계점 극복을 다른 지표들과 함께 적용해야함

### (2) 오차 행렬 (Confusion Matrix)
- 학습된 분류모델이 예측을 수행하면서 얼마나 confused되는지 보여주는 지표
- 이진분류의 예측오류가 얼마인지
- 어떤 유형의 예측 오류가 발생하고 있는지 나타내는 지표
- Confusion_matrix() API제공
- 불균형한 dataset에서는 positive데이터 건수가 매우 작아서 Negative로 예측 정확도가 높아지는 경향 있음



### (3) 정밀도 (Precision) 와 재현률 (Recall) 
- 불균형한 dataset에서 더 선호되는 평가 지표
- Positive dataset의 예측 성능에 좀 더 초점 맞춘 평가 지표
- 이진분류 모델의 업무 특성에 따라 특정 평가 지표가 더 중요하게 간주
    - 재현률이 정밀도보다 상대적으로 중요한 업무가 더 많음 
- 가장 좋은 성능 평가는 재현율 & 정밀도 모두 높은 수치 가지는 것
<br>

</br>

#### 정밀도
```
정밀도 = TP / (FP + TP)
```
- 예측을 Positive로 한 대상 중에 예측과 실제값이 Positive로 일치한 데이터의 비율 의미
- Positive 예측 성능을 더 정밀하게 측정
- prcision_score() 사용
- Ex1 ) 스펨메일 여부 판단 모델
    - 실제 Positive인 것을 Negative로 판단하면 사용자가 불편함 느끼는 정도임
    - 반면 실제 Negative인 것을 Positive로 판단하면 메일을 아예 받지 못해서 업무에 차질 생김

#### 재현률 
```
재현률 = TP / (FN + TP)
```
- 실제값이 Positive인 대상 중 예측과 실제값이 Positive로 일치한 데이터의 비율 의미 
- 민감도 (Sensitivity) & TPR ( True Positive Rate)라고 불림 
- recall_score() 사용
- Ex1 ) 암판단 모델은 재현률이 더 중요!
    - 실제 Positive인 환자를 Negative로 판단하면 큰일남
    - 반면 Negative인 환자를 Positive로 판단하면 별 문제 없음
- Ex2 ) 금융 사기 적발 모델
    - 실제 Positive인 건을 Negative로 판단하면 손해가 커짐
    - 반면 Negative인 건을 Positive로 판단하면 재검증 절차만을 가짐

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [2]:
def get_clf_eval(y_test, pred):
    # 오차행렬
    confusion=confusion_matrix(y_test,pred)
    # 정확도
    accuracy=accuracy_score(y_test, pred)
    # 정밀도
    precision=precision_score(y_test, pred)
    # 재현률
    recall=recall_score(y_test,pred)
    print(confusion)
    print(accuracy,precision,recall)

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [12]:
# 원본 데이터 재로딩, 데이터 가공, 학습 데이터 / 테스트 데이터 분할
titanic=pd.read_csv('C:/Users/admin/dataset/titanic_train.csv')

In [13]:
# null처리
def fillnan(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

In [14]:
# 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name', 'Ticket'], axis=1,inplace=True)
    return df

In [15]:
# Label Encoding 수행
def format_features(df):
    df['Cabin']=df['Cabin'].str[:1]
    features=['Cabin', 'Sex','Embarked']
    for f in features:
        le=preprocessing.LabelEncoder()
        le=le.fit(df[f])
        df[f]=le.transform(df[f])
    return df

In [16]:
# 앞에서 설정한 데이터 전처리 함수 호출( 전체 )
def transform_features(df):
    df=fillnan(df)
    df=drop_features(df)
    df=format_features(df)
    return df

In [17]:
y_df=titanic['Survived']
x_df=titanic.drop('Survived', axis=1)
x_df=transform_features(x_df)

In [18]:
X_train,X_test, y_train,y_test=train_test_split(x_df,y_df, test_size=0.20, random_state=11)

In [19]:
lrclf=LogisticRegression()

In [20]:
lrclf.fit(X_train, y_train)
pred=lrclf.predict(X_test)
get_clf_eval(y_test,pred)

[[108  10]
 [ 14  47]]
0.8659217877094972 0.8245614035087719 0.7704918032786885
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Result
```
오차행렬 => [[108  10]
             [ 14  47]]
정확도 => 0.8659217877094972 
정밀도 => 0.8245614035087719 
재현율 => 0.7704918032786885
```
정밀도에 비해 낮은 재현율 가짐

#### 정밀도 / 재현율 트레이드 오프 (Trade-off)
- 업무 특성상 정밀도 & 재현률이 강조되어야 하는 경우에 분류의 결정 임계값(Threshold) 조정해야함
- 정밀도 & 재현율은 상호보완적이기 때문에 하나를 높이면 다른 하나는 떨어짐
- 이를 정밀도 /재현율의 트레이드오프(Trade-off)라 함



### predict_proba()
- 개별 데이터별로 예측 확률을 반환 
- 학습이 완료된 Classifier 객체에서 호출 가능
- test feature 데이터 세트를 파라미터로 입력해주면 test feature 레코드의 개별 클래스 예측 확률을 반환